In [ ]:
## Import for data generation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
ID_21659081 = np.load("C:\\Users\\nico\\Desktop\\Stanford\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2019_Fires\\apr\\multi_day\\21659081.npy")

In [2]:
x_train_list = []
y_train_list = []
n_future = 1
n_past = 3

def nn_train_test(path, n_future, n_past):
    for file in os.listdir(r"{path}".format(path = path)):
        if os.path.getsize("{path}\\{file}".format(path = path, file = file)) < 100000:
            continue

        training_fire = np.load("{path}\\{file}".format(path = path, file = file))
        shape = np.array(training_fire.shape) 

        if shape[0] < n_past + n_future:
            continue

        standard_fire = np.zeros((len(training_fire), 256, 256))    
        if shape[1] > 255 or shape[2] > 255:
            standard_fire[:shape[0], :min(256, shape[1]), :min(256, shape[2])] = training_fire[:, :256, :256]
        else: 
            standard_fire[:shape[0], :shape[1], :shape[2]] = training_fire

        x_train = np.zeros((n_past, 256, 256))
        y_train = np.zeros((256, 256))


        for i in range(0 , len(standard_fire) - n_future - n_past + 1):
            x_train_list.append(standard_fire[i : i + n_past])
            y_train_list.append(standard_fire[i + n_past: i + n_past + n_future])

    x_train = np.array(x_train_list)
    y_train = np.array(y_train_list)

    x_train = np.reshape(x_train, (1, x_train.shape[1], x_train.shape[0], x_train.shape[2], x_train.shape[3]))

    y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[2], y_train.shape[3]))
    
    return x_train,y_train



    

In [ ]:
path = "C:\\Users\\nico\\Desktop\\Stanford\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2018_Fires\\feb\\multi_day"
x_train, y_train  = nn_train_test(path, 1, 3)

In [ ]:
print(x_train.shape, y_train.shape)

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf

In [4]:
from pyspark import SparkContext
#sc =SparkContext()

In [ ]:
#### WORKS! for the day before only though? 
regressor = Sequential()
print(x_train.shape)
dropout = 0.1
regressor.add(tf.keras.layers.Bidirectional(LSTM(units=n_past, return_sequences=True, input_shape = x_train[0].shape)))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))
regressor.add(Dense(units = n_future, activation = 'linear'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])

regressor.fit(x_train[0][2], y_train, epochs = 10, batch_size = 3)

In [16]:
x_train_list = []
y_train_list = []
n_future = 1
n_past = 3

def nn_train_test(path, n_future, n_past):
    for file in os.listdir(r"{path}".format(path = path)):
        if os.path.getsize("{path}\\{file}".format(path = path, file = file)) < 100000:
            continue

        training_fire = np.load("{path}\\{file}".format(path = path, file = file))
        shape = np.array(training_fire.shape) 

        if shape[0] < n_past + n_future:
            continue

        standard_fire = np.zeros((len(training_fire), 256, 256))    
        if shape[1] > 255 or shape[2] > 255:
            standard_fire[:shape[0], :min(256, shape[1]), :min(256, shape[2])] = training_fire[:, :256, :256]
        else: 
            standard_fire[:shape[0], :shape[1], :shape[2]] = training_fire
        
        standard_fire = np.reshape(standard_fire, (len(training_fire), 256**2))
        pca = decomposition.PCA(n_components=1)
        standard_fire = StandardScaler().fit_transform(standard_fire)
        pca.fit_transform(standard_fire)
        
        
        x_train = np.zeros((n_past, 256**2))
        y_train = np.zeros((256**2))


        for i in range(0 , len(standard_fire) - n_future - n_past + 1):
            x_train_list.append(standard_fire[i : i + n_past])
            y_train_list.append(standard_fire[i + n_past: i + n_past + n_future])

    x_train = np.array(x_train_list)
    y_train = np.array(y_train_list)

    x_train = np.reshape(x_train, (x_train.shape[0],  x_train.shape[2], x_train.shape[1]))

    y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[2]))
    
    return x_train,y_train,standard_fire


In [17]:
path = "C:\\Users\\nico\\Desktop\\Stanford\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2018_Fires\\feb\\multi_day"
x_train, y_train,standard_fire  = nn_train_test(path, 1, 3)

In [18]:
standard_fire.shape

(7, 65536)

In [7]:
x_train.shape, y_train.shape

((193, 65536, 3), (193, 65536))

In [8]:
from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

In [ ]:
regressor = Sequential()
print(x_train[0].shape)
dropout = 0.1
regressor.add(tf.keras.layers.Bidirectional(LSTM(units=n_past, return_sequences=True, input_shape = x_train[0].shape)))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))
'''
regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))

regressor.add(LSTM(units = n_past, return_sequences = True))
regressor.add(Dropout(dropout))
'''
regressor.add(Dense(units = n_future, activation = 'linear'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])

regressor.fit(x_train, y_train, epochs = 10, batch_size = 1)

In [ ]:
path = "C:\\Users\\nico\\Desktop\\Stanford\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2018_Fires\\mar\\multi_day"
x_test, y_test  = nn_train_test(path, 1, 3)


In [ ]:
y_test.shape

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range=(0,1))
print(x_test.shape)

#training_set_scaled = sc.fit_transform(x_train)

predict_train = regressor.predict(x_test[0][0])


#predict_train = np.reshape(predict_train, (7, 235))
#sc = MinMaxScaler(feature_range=(0,1))
#training_set_scaled = sc.fit_transform(predict_train)


#predicted_fires = sc.inverse_transform(predict_train)




print(predict_train.shape)


In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range=(0,1))

for i in range(4):
    for j in range(4):
        if i == j:
            continue
        for k in range(4):
            if i == k or j == k:
                continue
            for l in range(4):
                if i == l or j == l or k == l:
                    continue
                else:
                    try: 

                        x_test.reshape((x_test.shape[i], x_test.shape[j], x_test.shape[k], x_test.shape[l]))
                        predict_test = regressor.predict(x_test)
                        print("Ordering ({i}, {j}, {k}, {l}) IS feasible".format(i=x_test.shape[i], j=x_test.shape[j], k=x_test.shape[k], l=x_test.shape[l]))

                    except Exception as e:
                        print("Ordering ({i}, {j}, {k}, {l}) not feasible".format(i=x_test.shape[i], j=x_test.shape[j], k=x_test.shape[k], l=x_test.shape[l]))




#training_set_scaled = sc.fit_transform(x_train)




#predict_test = np.reshape(predict_train, (7, 235))
#sc = MinMaxScaler(feature_range=(0,1))
#training_set_scaled = sc.fit_transform(predict_train)


#predicted_fires = sc.inverse_transform(predict_train)



In [ ]:
training_set = ID_21659081
x_train = []
y_train = []

n_future = 1
n_past = 5

for i in range(0 , len(training_set) - n_future - n_past + 1):
    x_train.append(training_set[i : i + n_past])
    y_train.append(training_set[i + n_past: i + n_past + n_future])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[1], x_train.shape[0], x_train.shape[2], x_train.shape[3]))

y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[2], y_train.shape[3]))

